In [ ]:
import json

with open('data/rpfs_pos/USpowerGrid.json') as f:
    jsondata = json.load(f)


In [ ]:
import numpy as np


tq = 'stress'

data = []
indices = []


for i,e in enumerate(jsondata['e']):
    params = e['params']
    qs = []
    n_len = None
    for seed in e['seed']:
        t = e['seed'][seed]
        qs.append(t[tq])
        n_len = len([p for p in t['pos']])

    data.append([np.mean(np.array(qs)), params['edge_length'], params['number_of_pivots'], params['number_of_iterations'], params['eps']])
    # data.append([np.mean(np.array(qs)), (params['edge_length'] - 1) / (100 - 1), (params['number_of_pivots'] - 1) / (n_len - 1), (params['number_of_iterations'] - 1) / (100 - 1), (params['eps'] - 0.01)/(1 - 0.01)])
    indices.append(i)


In [ ]:
xs = []
ys = []

for d in data:
    xs.append(d[4])
    ys.append(d[0])

In [ ]:
import matplotlib.pyplot as plt


plt.scatter(xs, ys)

plt.show()


In [83]:
import json
import argparse

import optuna
import networkx as nx

from utils import graph_preprocessing, generate_graph_from_nx_graph, draw_graph, edge_crossing_finder
from quality_metrics import angular_resolution, aspect_ratio, crossing_angle, crossing_number, gabriel_graph_property, ideal_edge_length, node_resolution, shape_based_metrics, stress


def objective_wrapper(nx_graph, graph, indices, qnames, all_shortest_paths, edge_weight=1):
    def objective(trial: optuna.Trial):
        params = {
            'edge_length': trial.suggest_int('edge_length', 1, 100),
            'number_of_pivots': trial.suggest_int('number_of_pivots', 1, len(nx_graph.nodes)),
            'number_of_iterations': trial.suggest_int('number_of_iterations', 1, 1000),
            'eps': trial.suggest_float('eps', 0.01, 1)
        }
        trial.set_user_attr('params', params)

        pos = draw_graph(graph, indices, params)
        trial.set_user_attr('pos', pos)

        # quality_metrics = calc_quality_metrics(
        #     nx_graph, pos, all_shortest_paths, edge_weight)
        quality_metrics = calc_qs(
            nx_graph, pos, all_shortest_paths, qnames, edge_weight)
        trial.set_user_attr('quality_metrics', quality_metrics)

        result = tuple([quality_metrics[name]
                       for name in qnames])
        return result
    return objective


def parse_args():
    # lib/egraph-rs/js/dataset/{dataset_name}.json
    # data/optimization/{dataset_name}/{target_qs}.json
    parser = argparse.ArgumentParser()
    # USpowerGrid
    parser.add_argument('dataset_name')

    # node_resolution,stress or all
    parser.add_argument('target_qs')

    # target parameter
    # parser.add_argument('target_params')

    args = parser.parse_args()

    return args


def calc_qs(nx_graph, pos, all_shortest_paths, qnames, edge_weight=1):
    result = {}
    edge_crossing = None
    if 'crossing_angle' in qnames or 'crossing_number' in qnames:
        edge_crossing = edge_crossing_finder(nx_graph, pos)

    for qname in qnames:
        if qname == 'angular_resolution':
            result[qname] = angular_resolution.quality(nx_graph, pos)
        elif qname == 'aspect_ratio':
            result[qname] = aspect_ratio.quality(pos)
        elif qname == 'crossing_angle':
            result[qname] = crossing_angle.quality(
                nx_graph, pos, edge_crossing)
        elif qname == 'crossing_number':
            result[qname] = crossing_number.quality(
                nx_graph, pos, edge_crossing)
        elif qname == 'gabriel_graph_property':
            result[qname] = gabriel_graph_property.quality(nx_graph, pos)
        elif qname == 'ideal_edge_length':
            result[qname] = ideal_edge_length.quality(nx_graph, pos)
        elif qname == 'node_resolution':
            result[qname] = node_resolution.quality(pos)
        elif qname == 'shape_based_metrics':
            result[qname] = shape_based_metrics.quality(
                nx_graph, pos, edge_weight)
        elif qname == 'stress':
            result[qname] = stress.quality(nx_graph, pos, all_shortest_paths)

    return result


EDGE_WEIGHT = 50
N_TRIALS = 100

all_qnames = [
    'angular_resolution',
    'aspect_ratio',
    'crossing_angle',
    'crossing_number',
    'gabriel_graph_property',
    'ideal_edge_length',
    'node_resolution',
    'shape_based_metrics',
    'stress']
qmap = {
    'angular_resolution': angular_resolution,
    'aspect_ratio': aspect_ratio,
    'crossing_angle': crossing_angle,
    'crossing_number': crossing_number,
    'gabriel_graph_property': gabriel_graph_property,
    'ideal_edge_length': ideal_edge_length,
    'node_resolution': node_resolution,
    'shape_based_metrics': shape_based_metrics,
    'stress': stress
}
base_params = {
    'edge_length': 30,
    'number_of_pivots': 50,
    'number_of_iterations': 100,
    'eps': 0.1
}

# args = parse_args()
dataset_name = 'bull'
# target_qs = args.target_qs
# target_params = args.target_params

# if target_params not in base_params:
#     raise ValueError(f'{target_params}は無効です')

dataset_path = f'lib/egraph-rs/js/dataset/{dataset_name}.json'
# export_path = f'data/find_limitations/{dataset_name}/{target_qs}.json'

# target_qnames = [
#     qname for qname in all_qnames] if target_qs == 'all' else target_qs.split(',')

# qnames = []
# for tqname in target_qnames:
#     if tqname in all_qnames:
#         qnames.append(tqname)
#     else:
#         raise ValueError(f'{tqname} in {target_qnames} is not accepted')
# qnames = sorted(qnames)

with open(dataset_path) as f:
    graph_data = json.load(f)
nx_graph = graph_preprocessing(nx.node_link_graph(graph_data), EDGE_WEIGHT)
graph, indices = generate_graph_from_nx_graph(nx_graph)
all_shortest_paths = dict(nx.all_pairs_dijkstra_path_length(nx_graph))

pos = draw_graph(graph, indices, base_params, 0)

print(graph)


In [87]:
nx_graph.edges(data=True)

EdgeDataView([('0', '1', {'weight': 50, 'id': '0'}), ('0', '2', {'weight': 50, 'id': '1'}), ('1', '2', {'weight': 50, 'id': '2'}), ('1', '3', {'weight': 50, 'id': '3'}), ('2', '4', {'weight': 50, 'id': '4'})])

In [84]:
for a,b in nx_graph.edges:
    print((a,b))

('0', '1')
('0', '2')
('1', '2')
('1', '3')
('2', '4')


In [85]:
print(graph.edge_weight(0))
print(graph.edges(0))
print(graph.edges(1))
print(graph.edge_indices())


('0', '1')
[('0', '2'), ('0', '1')]
[('1', '3'), ('1', '2')]
[0, 1, 2, 3, 4]


In [86]:
graph.add_edge()

TypeError: Graph.add_edge() missing 3 required positional arguments: 'a', 'b', and 'value'

In [124]:
for iter_n in range(1, 20 + 1):
    params_v = 10 ** -(1.1 ** (iter_n - 1) - 1)
    print(params_v)

1.0
0.7943282347242814
0.6165950018614819
0.4666593803142882
0.3434788498657801
0.24518279977335575
0.16921505521851227
0.11253377817846805
0.0718474220708564
0.04385835201909756
0.02548340990009201
0.014024367830448045
0.007270622954918933
0.003529626777994801
0.0015940489889168697
0.0006648931069620805
0.0002541130762044737
8.821241561819024e-05
2.7547531383556598e-05
7.657569641252238e-06
